In [2]:
library(tidyverse)
library(magrittr) # better handling of pipes
library(purrr) # to work with lists and map functions
library(glue) # to paste strings
library(stringr) # to hand strings
library(rvest) # rvest makes scraping easier
library(polite) # polite is the "polite" version of rvest
library(xml2) # makes it easier to work with HTML and XML from R
library(ggplot2)
library(httr)

In [4]:
HH_LC_PI_df <- read_csv(file = "data/household-living-costs-price-indexes-June-2021-quarter-time-series-indexes.csv")

Rows: 45262 Columns: 11

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (8): hlpi_name, series_ref, quarter, hlpi, nzhec, nzhec_name, nzhec_shor...
dbl (3): index, change.q, change.a


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



In [28]:
all_households_petrol <- HH_LC_PI_df %>%
    filter(hlpi_name == "All households", nzhec_name == "Petrol")
all_households_petrol %>% head

hlpi_name,series_ref,quarter,hlpi,nzhec,nzhec_name,nzhec_short,level,index,change.q,change.a
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
All households,HPIQ.SALL07202,2008Q2,allhh,7.2.02,Petrol,Petrol,class,915,NA,NA
All households,HPIQ.SALL07202,2008Q3,allhh,7.2.02,Petrol,Petrol,class,957,4.6,NA
All households,HPIQ.SALL07202,2008Q4,allhh,7.2.02,Petrol,Petrol,class,743,-22.4,NA
All households,HPIQ.SALL07202,2009Q1,allhh,7.2.02,Petrol,Petrol,class,736,-0.9,NA
All households,HPIQ.SALL07202,2009Q2,allhh,7.2.02,Petrol,Petrol,class,760,3.3,-16.9
All households,HPIQ.SALL07202,2009Q3,allhh,7.2.02,Petrol,Petrol,class,775,2.0,-19.0


In [27]:
all_households_rent <- HH_LC_PI_df %>%
    filter(hlpi_name == "All households", nzhec_short == "Rent", level == "subgroup")
all_households_rent %>% head

hlpi_name,series_ref,quarter,hlpi,nzhec,nzhec_name,nzhec_short,level,index,change.q,change.a
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
All households,HPIQ.SALL041,2008Q2,allhh,4.1,Actual rentals for housing,Rent,subgroup,895,NA,NA
All households,HPIQ.SALL041,2008Q3,allhh,4.1,Actual rentals for housing,Rent,subgroup,902,0.8,NA
All households,HPIQ.SALL041,2008Q4,allhh,4.1,Actual rentals for housing,Rent,subgroup,906,0.4,NA
All households,HPIQ.SALL041,2009Q1,allhh,4.1,Actual rentals for housing,Rent,subgroup,908,0.2,NA
All households,HPIQ.SALL041,2009Q2,allhh,4.1,Actual rentals for housing,Rent,subgroup,910,0.2,1.7
All households,HPIQ.SALL041,2009Q3,allhh,4.1,Actual rentals for housing,Rent,subgroup,912,0.2,1.1


In [29]:
all_households_food <- HH_LC_PI_df %>%
    filter(hlpi_name == "All households", nzhec_short == "Food")
all_households_food %>% head

hlpi_name,series_ref,quarter,hlpi,nzhec,nzhec_name,nzhec_short,level,index,change.q,change.a
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
All households,HPIQ.SALL01,2008Q2,allhh,1,Food,Food,group,870,NA,NA
All households,HPIQ.SALL01,2008Q3,allhh,1,Food,Food,group,903,3.8,NA
All households,HPIQ.SALL01,2008Q4,allhh,1,Food,Food,group,918,1.7,NA
All households,HPIQ.SALL01,2009Q1,allhh,1,Food,Food,group,927,1.0,NA
All households,HPIQ.SALL01,2009Q2,allhh,1,Food,Food,group,935,0.9,7.5
All households,HPIQ.SALL01,2009Q3,allhh,1,Food,Food,group,949,1.5,5.1


In [16]:
HH_LC_PI_df %>%
    filter(hlpi_name == "All households") %>%
    group_by(nzhec_name) %>%
    summarise()

nzhec_name
<chr>
Accommodation services
Actual rentals for housing
Alcoholic beverages
Alcoholic beverages and tobacco
All groups
Audio-visual and computing equipment
Cigarettes and tobacco
Clothing
Clothing and footwear
